<h1><i> Final Capstone Project </i></h1>

<h4> 1. Description problem </h4>

<p> In this final project we will help to an Bank to evaluate where can open in New York. </br>
Recently the bank is operating in Toronto. For the Executive Directory it's important know how the City of New York is clustering, looking for the principal businnes are exits in Manhattan.</p>

<p> Based on the New York Neighbourhoods, we have to explore and find some similarys Neighbourhoods, compared with Toronto. </p>

<h4> 2. About the Data </h4>

<h4> 2.1 New York Dataset </h4>

<p> First we will download the dataset from, <a href="https://geo.nyu.edu/catalog/nyu_2451_34572">New York: A City of Neighborhoods.</a>, where is currently operating the bank </p>

In [109]:
import json

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
    
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)   

neighborhoods_data = newyork_data['features']
df_n = pd.DataFrame()

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_n = df_n.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

df_n = df_n[['Borough','Neighborhood','Latitude','Longitude']]    
df_n.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<p> Then we put the data on a Folium Map to visualize the Neighborhoods </p>

In [110]:
import folium
from geopy.geocoders import Nominatim

manhattan_data = df_n[df_n['Borough'] == 'Manhattan'].reset_index(drop=True)
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<h4> 2.2 Toronto Dataset </h4>

<p> In second place, we need the neighbourhoods from Toronto, so for that we are going to download first, the postal code of each neighbourhood from Wikipedia through Web Scraping using BeautifulSoup libraries.</p>

In [111]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

postal_codes = []

req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "html.parser")

postal_table = soup.find('table',{"class":"wikitable sortable"})

for row in postal_table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        postal_codes.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))

df_t = pd.DataFrame(postal_codes)
df_t.columns = ['Postcode', 'Borough','Neighbourhood']
df_t = df_t[df_t.Borough != 'Not assigned']
df_t = df_t.rename(columns={'Postcode': 'PostalCode'})

df_t.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<p> Then we JOIN the Toronto dataset with Geospatial data obtained from Cognitive Class.</p>
<p> And, same as New York, we put the data on a Folium Map to visualize the Neighborhoods </p>

In [112]:
!wget -q -O 'canada_data.csv' https://cocl.us/Geospatial_data
from geopy.geocoders import Nominatim 
import folium 

df_coordinates = pd.read_csv("canada_data.csv") 
df_coordinates = df_coordinates.rename(columns={'Postal Code': 'PostalCode'})
df_coordinates = pd.merge(df_t, df_coordinates, on='PostalCode', how='inner')
df_coordinates = df_coordinates[['Borough','Neighbourhood','Latitude','Longitude']]    

toronto_data = df_coordinates[df_coordinates['Borough'].str.contains('Toronto')]

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_canada = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

<h4> 3.0 Brief summary </h4>

In [113]:
print(len(df_n['Borough'].unique()))
print(len(df_t['Borough'].unique()))

5
11


<p> A brief summary: We have 306 rows in the data set from New York and from Toronto there are 212 rows. </p>
<p> And New York has 5 Borough and Toronto 11 </p>

<h4> 4. Methodology </h4>

<p> We will use the Foursquare API to get the nearby venues, and k-means clustering algorithm to analyze the Neighbourhood </p>